# Demonstrating *"chaining"* the binomial option pricing method over an arbitrary timeframe to show how it can approximate Black-Scholes

Consider the following scenario. 

![Example Scenario](img/two_period_example.png)

## Question: What would be the fair price of European style call option that expires 1 year from now?

I'm going to things differently. The answer here is `$ 4.43`. But how did I get to this answer and how do I know that it is correct? Rather than just asking you to blindly trust me, I'll show you my thought process.

## Start with a naive assumption.

Consider answering this question using the single period binomial option pricing method? After all, we know the current stock price, the strike price, the payoffs in the up and down state, and risk free rate. Is it safe to assume that we can focus on just the beginning and the end? Like this? 

![A niave assumption](img/naive_assumption_example.png)

This seems like a promising start. Let's price it anyway using the binomial pricing code I built.

In [1]:
from atop.options.calloption import CallOption

call_calc = CallOption(100, 110, 121, 90.25, .1)
call_calc.print_calc_values()

  INFORMATION FOR A SINGLE PEIORD CALL OPTION USING BINMOMIAL PRICING METHOD

The calculations below are for a single period CALL option where the underlying value is 100, 
with a strike price of 110, an up price of 121, a down price of 90.25, and a risk free rate of 10.0%.
All outputs are rounded to 2 decimal places.  

--------------
Calculations
--------------
Calculated Option Price: 6.42
Calculated Hedge Ratio: 0.36
Calculated present value of bond position: -29.35
Up state payoff is 11 and down state payoff is 0
Calculated risk neutral probability for up state is 0.64 and for down state risk neutral probability is 0.36
_____________________________________________________________________________________________________________



Oh look an answer... it appears the price is ` $6.42`. But wait...

# Looks can be deceiving; *this answer is wrong!*

How do we know that it is wrong? Well, let's think about the assumptions we made here and see if they hold. The implicit assumptions here are first, there are only 2 payoff states and second, the option price doesn't evolve over time. 

## Is it safe to assume that there are only 2 payoff states?

Let's look back to the question. Are there only 2 end payoff states? What does the tree look like again?

![The tree](img/tree_example.png)

If we look all the way to the right we can see that there are actually 3 payoff states, not 2. Since this assumption failed then we know that the answer must be off. We could stop here if we wanted satisfied that `$ 6.42` isn't the correct price. But let's analyze the second assumption as it will become useful in a bit.

## Is it safe to assume that the option value doesn't evolve over time?

That is to say, suppose we paid some price, P for a call option and then the underlying price crashes into the ground such that it is so far low below the strike price that there is zero chance of recovering. Is that option still worth what we paid? NO! I guarentee that it's worthless. In fact if you find someone who is willing to pay anything near what you bought the call option for, please let me know, I've got a bridge I want to sell them. 

Conversely, what if the underlying price skyrockets well above the strike price and we go to sell our option? Is it still worth what we paid for? NO! At the very least, it should be worth the difference in the current underlying price and the strike price. This is especially true for American style call options and European call options that are close to expiration. If you sold it for anything less, then you are essentially giving free money away. 

Rephrased, suppose that we messed up and actually bought a call option at `$ 6.42` on this underlying asset (seriously, I've got a bridge for sale). Does this mean that we can never profit from this transaction? Well, in actuality, our mistake doesn't rule out the possiblity of profit. Which is good news. But it doesn't guarentee that we will make a profit (or fully recover the price we paid.)\*


\* Side question. If possible, at what point will the option price be higher than `$ 6.42`?

So are we on track to actually solve this problem? Suprisingly, yes. There is an unstated assumption we got correct; using the single period bionmial option pricing method. Think of the method first and foremost as a tool. Every tool has a time and place. "A screwdriver isn't useful as a hammer." Rather than retool our method, let's change where and when we use it.  

# A helpful visualization:
If you haven't noticed it yet then let me point it out; the tree appears to be several different options *chained* together. Let's seperate out these options. 

### Let's call this the 'up' option.
(Because in the prior period the underlying asset price went up)

![The 'Up' Option](img/up_option_example.png)

### Let's call this the 'down' option.

![The 'Down' Option](img/down_option_example.png)


### Lastly let's call this Today's option.

![The 'Starting' Option](img/start_option_example.png)

When we break out the original tree into distinct single period options, suddenly the tasks looks a lot more solvable. Let's try using the single period binomial option pricing now and see where that gets us.

# The final product:

In [2]:
up_opt = CallOption(110, 110, 121, 104.50, 0.05)
down_opt = CallOption(95, 110, 104.50, 90.25, 0.05)
start_opt = CallOption(100, 110, 110, 95, 0.05, up_opt.option_price, down_opt.option_price)
start_opt.print_calc_values()

  INFORMATION FOR A SINGLE PEIORD CALL OPTION USING BINMOMIAL PRICING METHOD

The calculations below are for a single period CALL option where the underlying value is 100, 
with a strike price of 110, an up price of 110, a down price of 95, and a risk free rate of 5.0%.
All outputs are rounded to 2 decimal places.  
USER HAS OPTED TO OVERRIDE PAYOFF VALUES! BE AWARE THIS WILL AFFECT CALCULATIONS!

--------------
Calculations
--------------
Calculated Option Price: 4.43
Calculated Hedge Ratio: 0.47
Calculated present value of bond position: -42.13
Up state payoff is 6.984126984126988 and down state payoff is 0
Calculated risk neutral probability for up state is 0.67 and for down state risk neutral probability is 0.33
_____________________________________________________________________________________________________________



# temp writing space.

Here's why. The single period binomial pricing model accurately prices an option for an *arbitrary* period of time. Notice that at no point in my code do I ask for a time to expiration date. One would assume that this was because the code could only be used for 1 year time periods. In fact, this is not the case. Additionally, the risk-free rate is *not* denoted as an annual risk-free rate but instead as simply, the risk free rate. Lastly, did you notice that each class has an override ability? By default, the code assumes it can calculate the payoffs from the difference in the strike price and the up and down values. So one would think you never need to override the payoffs. But that's not the case for multi-period 
